#### Collaborative Filtering Recommender System

In [2]:

import pandas as pd
import numpy as np
from sklearn.metrics import roc_auc_score, f1_score, accuracy_score, recall_score, precision_score

# Veri setini yükleme
data_path = './ml-latest-small/ratings.csv'  # Kendi dosya yolunuza göre güncelleyin
df = pd.read_csv(data_path)

# Kullanıcı-film puan matrisini oluşturma
rating_matrix = df.pivot_table(index='userId', columns='movieId', values='rating')

# Kullanıcılar arası benzerlik matrisini oluşturma (Pearson korelasyonu)
user_similarity = rating_matrix.T.corr()

# Kullanıcıya öneri yapma fonksiyonu
def predict_rating(user_id, movie_id, k=5):
    if movie_id not in rating_matrix.columns:
        return 3.0  # Film henüz puanlanmamışsa, ortalama bir puan döndür (varsayılan: 3.0)
    
    # Kullanıcının diğer tüm kullanıcılarla benzerlik skorlarını al
    sim_scores = user_similarity[user_id].drop(user_id).dropna()
    
    # Filmi puanlayan kullanıcılar arasından en benzer k kullanıcıyı seç
    movie_ratings = rating_matrix[movie_id].dropna()
    relevant_users = movie_ratings.index.intersection(sim_scores.index)
    sim_scores = sim_scores[relevant_users].sort_values(ascending=False)
    
    if sim_scores.empty:
        return 3.0  # Eğer benzer kullanıcı yoksa varsayılan bir puan döndür

    top_k_users = sim_scores.head(k)
    weighted_sum = sum(top_k_users * rating_matrix.loc[top_k_users.index, movie_id])
    similarity_sum = sum(top_k_users.abs())

    if similarity_sum == 0:
        return 3.0  # Benzerlik skoru toplamı sıfırsa, varsayılan puan döndür

    # Tahmini puanı döndür
    return weighted_sum / similarity_sum

# Test setini hazırlama
test_set = df.sample(frac=0.2, random_state=42)
train_set = df.drop(test_set.index)

# Tahminleri hesaplama ve değerlendirme
y_true = []
y_pred = []

for _, row in test_set.iterrows():
    user_id, movie_id, true_rating = int(row['userId']), int(row['movieId']), row['rating']
    predicted_rating = predict_rating(user_id, movie_id)
    
    # Gerçek ve tahmini puanları binary formatta karşılaştırmak için eşik değeri (threshold)
    threshold = 3.5
    y_true.append(1 if true_rating >= threshold else 0)
    y_pred.append(1 if predicted_rating >= threshold else 0)

# Değerlendirme metriklerini hesaplama
auc = roc_auc_score(y_true, y_pred)
f1 = f1_score(y_true, y_pred)
accuracy = accuracy_score(y_true, y_pred)
recall = recall_score(y_true, y_pred)
precision = precision_score(y_true, y_pred)

print(f'AUC: {auc:.4f}')
print(f'F1 Score: {f1:.4f}')
print(f'Accuracy: {accuracy:.4f}')
print(f'Recall: {recall:.4f}')
print(f'Precision: {precision:.4f}')


AUC: 0.7051
F1 Score: 0.7297
Accuracy: 0.6969
Recall: 0.6684
Precision: 0.8032


#### Content-Based Filtering Recommender System

In [4]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics import roc_auc_score, f1_score, accuracy_score, recall_score, precision_score

# Film ve puan veri setlerini yükleme
movies_path = './ml-latest-small/movies.csv'  # Kendi dosya yolunuza göre güncelleyin
ratings_path = './ml-latest-small/ratings.csv'

movies_df = pd.read_csv(movies_path)
ratings_df = pd.read_csv(ratings_path)

# Genre (tür) metinlerini TF-IDF vektörlerine dönüştürme
tfidf_vectorizer = TfidfVectorizer(stop_words='english')
tfidf_matrix = tfidf_vectorizer.fit_transform(movies_df['genres'])

# Film benzerlik matrisini hesaplama (cosine similarity)
cosine_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)

# Kullanıcı profiline göre öneri yapma fonksiyonu
def predict_rating(user_id, movie_id, k=5):
    user_ratings = ratings_df[ratings_df['userId'] == user_id]
    if movie_id not in movies_df['movieId'].values:
        return 3.0  # Film bulunamadıysa varsayılan bir puan döndür

    # Kullanıcının daha önce puanladığı filmler ve bu filmlere verdiği puanlar
    user_rated_movie_indices = movies_df[movies_df['movieId'].isin(user_ratings['movieId'])].index
    movie_index = movies_df[movies_df['movieId'] == movie_id].index[0]

    # Kullanıcının puanladığı filmlerle tahmin yapılacak film arasındaki benzerlik skorlarını al
    sim_scores = cosine_sim[movie_index, user_rated_movie_indices]

    if sim_scores.sum() == 0:
        return 3.0  # Benzerlik skoru yoksa varsayılan bir puan döndür

    # Kullanıcının profiline göre filmin tahmini puanını hesapla
    weighted_sum = sum(sim_scores * user_ratings.set_index('movieId').loc[movies_df.loc[user_rated_movie_indices, 'movieId']]['rating'])
    similarity_sum = sum(sim_scores)

    return weighted_sum / similarity_sum

# Test setini hazırlama
test_set = ratings_df.sample(frac=0.2, random_state=42)
train_set = ratings_df.drop(test_set.index)

# Tahminleri hesaplama ve değerlendirme
y_true = []
y_pred = []

for _, row in test_set.iterrows():
    user_id, movie_id, true_rating = int(row['userId']), int(row['movieId']), row['rating']
    predicted_rating = predict_rating(user_id, movie_id)

    # Gerçek ve tahmini puanları binary formatta karşılaştırmak için eşik değeri (threshold)
    threshold = 3.5
    y_true.append(1 if true_rating >= threshold else 0)
    y_pred.append(1 if predicted_rating >= threshold else 0)

# Değerlendirme metriklerini hesaplama
auc = roc_auc_score(y_true, y_pred)
f1 = f1_score(y_true, y_pred)
accuracy = accuracy_score(y_true, y_pred)
recall = recall_score(y_true, y_pred)
precision = precision_score(y_true, y_pred)

print(f'AUC: {auc:.4f}')
print(f'F1 Score: {f1:.4f}')
print(f'Accuracy: {accuracy:.4f}')
print(f'Recall: {recall:.4f}')
print(f'Precision: {precision:.4f}')


AUC: 0.6933
F1 Score: 0.7295
Accuracy: 0.6907
Recall: 0.6817
Precision: 0.7845
